# Make depth summary figures

This notebook is to produce some figures for the ELAIS-N1 paper.

First we will, for every helpix 13 in HELP get the highest K or Ks depth, g depth, and irac 1 depth. We will then plot cumulative area histograms and generate the 25%, 50%, 75% areas to summarise the depths across all HELP.



In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))


This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000)


In [2]:
import pyvo as vo
import glob
import time
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import random

import herschelhelp as hh
from herschelhelp_internal.utils import flux_to_mag

from astropy.table import Table, Column, vstack, join, unique

from pymoc import MOC

import pandas as pd

import yaml

from pcigale.sed import SED
from pcigale.sed_modules import get_module

In [3]:

#Then we establish the VO connection to our database
service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap")

In [4]:
herschelhelp_python_loc = ('../../../../herschelhelp_python/')

filter_locs  = glob.glob(herschelhelp_python_loc + 'database_builder/filters/*')

In [5]:
filters = [f.split('/')[-1].split('.')[0] for f in filter_locs]
try:
    filters.remove('readme')
except ValueError:
    print('no readme')

bands = ['u', 'g', 'r', 'i', 'z', 'y',
        'J', 'H', 'K', 'Ks',
        'i1', 'i2', 'i3', 'i4']

no readme


In [6]:
filters

['wfi_696nm',
 'suprime_nb816',
 'wfc3_f125w',
 'mmt_g',
 'omegacam_r',
 'decam_z',
 'wfi_416nm',
 'sdss_i',
 'omegacam_g',
 'suprime_ib464',
 'wfi_571nm',
 'mmt_r',
 'wfc3_f105w',
 'spire_350',
 'decam_y',
 'suprime_ib505',
 'cfht12k_b',
 '90prime_z',
 'omegacam_u',
 'mips_160',
 'suprime_ip',
 'isaac_k',
 'galex_nuv',
 'acs_f775w',
 'moircs_k',
 'sdss_z',
 'acs_f606w',
 'cfht12k_r',
 'decam_i',
 'quirc_hk',
 'mmt_u',
 'suprime_zpp',
 'suprime_ia624',
 'megacam_r',
 'bessell_v',
 'wircam_ks',
 'mips_24',
 'omega2000_j',
 'wfi_r',
 'spire_250',
 'suprime_r',
 'pacs_blue',
 'gpc1_i',
 'suprime_g',
 'bessell_b',
 'bessell_u',
 'mosaic_r',
 'wfi_753nm',
 'acs_f850lp',
 'ukidss_y',
 'megacam_g',
 'wfc_z',
 'lbc_u',
 'suprime_ia527',
 'wfc_i',
 'wfi_b123',
 'gpc1_y',
 'suprime_b',
 'wfi_b',
 'wfi_u',
 'suprime_ia679',
 'ukidss_j',
 'suprime_ib827',
 'suprime_ia484',
 'ukidss_k',
 'megacam_u',
 'newfirm_j1',
 'suprime_n816',
 'gpc1_z',
 'newfirm_j3',
 'suprime_v',
 'wfi_v',
 'mosaic_b',
 'mo

In [7]:
irac_i1_query="""
SELECT DISTINCT hp_idx_O_10, ferr_ap_irac_i1_mean
FROM depth.main
WHERE ferr_ap_irac_i1_mean IS NOT NULL
"""

#Then we execute the query
#resultset = service.run_async(irac_i1_query)
job = service.submit_job(irac_i1_query)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(10) #wait ten seconds and try again
    
table = job_result.fetch_result() 
i1_table = table.table

Job still running after 0 seconds.
Job still running after 10 seconds.
Job still running after 20 seconds.


AttributeError: 'NoneType' object has no attribute 'href'

In [ ]:
i1_table

In [ ]:
plt.hist(np.log10(i1_table['ferr_ap_irac_i1_mean']) )

In [ ]:
irac_i1_moc = MOC(10, i1_table['hp_idx_o_10'])
irac_i1_moc.area_sq_deg
total_area = irac_i1_moc.area_sq_deg
total_area

In [ ]:
fig, ax = plt.subplots()

cells, fluxes = np.histogram(np.log10(i1_table['ferr_ap_irac_i1_mean']*1.e-6), bins=50)
ax.plot(fluxes[1:],
         np.cumsum(cells)*total_area/cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('Log$_{10}$ ( Mean IRAC 1 flux error [Jy] )')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-6.5,-3.0])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.4 * column_width_cm
hieght_cm = width_cm / 1.618
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.tight_layout()
plt.savefig('./figs/IRAC_i1_cumulative_area_depth.pdf')
plt.savefig('./figs/IRAC_i1_cumulative_area_depth.png')

In [ ]:
i1_p25, i1_p50, i1_p75 = np.nanpercentile(i1_table['ferr_ap_irac_i1_mean'] * 3.,[25, 50, 75])
"""
The IRAC i1 band has coverage over {} square degrees with  3 sigma 
depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.""".format(total_area, i1_p25, i1_p50, i1_p75)

In mags as per reviewer request

In [ ]:
fig, ax = plt.subplots()

i1_pix_mag_depths, _ = flux_to_mag(i1_table['ferr_ap_irac_i1_mean']*1.e-6*5)
cells, fluxes = np.histogram(i1_pix_mag_depths, bins=1000)
#We want cumulative from faint to bright
cells =np.flip(cells)
fluxes = np.flip(fluxes)
ax.plot(fluxes[1:],
         np.cumsum(cells)*total_area/cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('5$\sigma$ IRAC $i1$ depth [mag]')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([23,16])
ax.set_ylim([0,300])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)



plt.savefig('./figs/IRAC_i1_cumulative_area_depth_mag.pdf', bbox_inches='tight')
plt.savefig('./figs/IRAC_i1_cumulative_area_depth_mag.png', bbox_inches='tight')

In [ ]:
mi1_p25, mi1_p50, mi1_p75 = np.nanpercentile(i1_pix_mag_depths ,[25, 50, 75])
"""
The IRAC i1 band has coverage over {} square degrees with  5 sigma 
depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.""".format(total_area, mi1_p25, mi1_p50, mi1_p75)

## g bands

There are multiple g type bands so we the the maximum depth of all of them.

In [ ]:
g_filters = [f for f in filters if f.split('_')[1] == 'g']
spaced_list = ''
for g in g_filters:
    spaced_list = spaced_list + ', ferr_ap_' + g + '_mean'
print(spaced_list)

In [ ]:
g_bands_query="""
SELECT DISTINCT hp_idx_O_10{}
FROM depth.main

""".format(spaced_list)
g_bands_query

In [ ]:
#Then we execute the query
#resultset = service.run_async(irac_i1_query)
job = service.submit_job(g_bands_query)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(10) #wait ten seconds and try again
    
print(job.phase)
table = job_result.fetch_result() 




In [ ]:
g_depth_cols  = ["ferr_ap_{}_mean".format(band) for band in g_filters]
g_depth_cols

In [ ]:
g_table = Table(table.table)
for col in g_depth_cols:
    g_table[col].fill_value = np.nan
g_table = g_table.filled()

In [ ]:
np.nanmin([g_table[column] for column in g_depth_cols], axis=0)

In [ ]:
for col in g_depth_cols:
    print(col, np.sum(np.log10(np.array(g_table[col])*1.e-6) < -9))

In [ ]:
decam_low = (np.log10(np.array(g_table['ferr_ap_decam_g_mean'])*1.e-6) < -9)
prime_low = (np.log10(np.array(g_table['ferr_ap_90prime_g_mean'])*1.e-6) < -9)

In [ ]:
decam_bad_moc = MOC(10, g_table['hp_idx_o_10'][decam_low])
prime_bad_moc = MOC(10, g_table['hp_idx_o_10'][decam_low])

fields = yaml.load(open('../../../dmu2/meta_main.yml', 'r'))


for field in fields['fields']:
    field_moc = MOC(filename=field['region'].replace('dmu_products/', '../../../'))
    decam_bad_area = decam_bad_moc.intersection(field_moc).area_sq_deg
    prime_bad_area = prime_bad_moc.intersection(field_moc).area_sq_deg
    print("{} has {} sq deg of bad DECam and {} sq degrees of bad 90Prime".format(field['name'], 
                                                                                  decam_bad_area, 
                                                                                  prime_bad_area))

In [ ]:
g_table['ferr_ap_decam_g_mean'][decam_low] = g_table['ferr_ap_decam_g_mean'][decam_low] * 1.e6
g_table['ferr_ap_90prime_g_mean'][prime_low] = g_table['ferr_ap_90prime_g_mean'][prime_low] * 1.e6

In [ ]:
g_table.add_column(Column(data=np.nanmin([g_table[column] for column in g_depth_cols], axis=0), name='ferr_g_min'))

In [ ]:
g_table['hp_idx_o_10', 'ferr_g_min']

In [ ]:
g_moc = MOC(10, g_table['hp_idx_o_10'])
g_moc.area_sq_deg
total_area_g = g_moc.area_sq_deg
total_area_g

In [ ]:
g_table['hp_idx_o_10', 'ferr_g_min']

fig, ax = plt.subplots()

good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

cells, fluxes = np.histogram(np.log10(np.array(g_table['ferr_g_min'][good ])*1.e-6), bins=100)
ax.plot(fluxes[1:],
         np.cumsum(cells)*1270./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('Log$_{10}$ ( Mean g flux error [Jy] )')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-8,-4.0])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm / 1.618
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.tight_layout()
plt.savefig('./figs/g_cumulative_area_depth.pdf')
plt.savefig('./figs/g_cumulative_area_depth.png')

In [ ]:
g_p25, g_p50, g_p75 = np.nanpercentile(g_table['ferr_g_min'][good ] * 3.,[25, 50, 75])
"""
The g band has coverage over {} square degrees with  3 sigma 
depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.""".format(1270, g_p25, g_p50, g_p75)

In mags as per reviewer request

In [ ]:
g_table['hp_idx_o_10', 'ferr_g_min']

fig, ax = plt.subplots()

good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

g_pix_mag_depths, _ = flux_to_mag(np.array(g_table['ferr_g_min'][good ])*1.e-6*5)
cells, fluxes = np.histogram(g_pix_mag_depths, bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*1270./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('5$\sigma$ $g$ depth [mag] ')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([27,20])
ax.set_ylim([0,1300])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/g_cumulative_area_depth_mag.pdf', bbox_inches='tight')
plt.savefig('./figs/g_cumulative_area_depth_mag.png', bbox_inches='tight')

In [ ]:
mg_p25, mg_p50, mg_p75 = np.nanpercentile(g_pix_mag_depths,[25, 50, 75])
"""
The g band has coverage over {} square degrees with  3 sigma 
depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.""".format(1270, mg_p25, mg_p50, mg_p75)

In [ ]:
g_table[prime_low]

## K or Ks filters
the same for K

In [ ]:
K_filters = [f for f in filters if f.split('_')[1].startswith('k')]
K_filters

In [ ]:

spaced_list = ''
for K in K_filters:
    spaced_list = spaced_list + ', ferr_ap_' + K + '_mean'
print(spaced_list)

In [ ]:
K_bands_query="""
SELECT DISTINCT hp_idx_O_10{}
FROM depth.main

""".format(spaced_list)
K_bands_query

In [ ]:
#Then we execute the query
#resultset = service.run_async(irac_i1_query)
job = service.submit_job(K_bands_query)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(10) #wait ten seconds and try again
    
print(job.phase)
table = job_result.fetch_result() 

In [ ]:
depth_cols  = ["ferr_ap_{}_mean".format(band) for band in K_filters]
depth_cols

In [ ]:
K_table = Table(table.table)
for col in depth_cols:
    K_table[col].fill_value = np.nan
K_table = K_table.filled()

In [ ]:
len(K_table)

In [ ]:
K_table.add_column(Column(data=np.nanmin([K_table[column] for column in depth_cols], axis=0), name='ferr_K_min'))

In [ ]:
np.sum(np.isnan(K_table['ferr_K_min'] ))

In [ ]:
K_moc = MOC(10, K_table[~np.isnan(K_table['ferr_K_min'] )]['hp_idx_o_10'])
K_moc.area_sq_deg
total_area_K = K_moc.area_sq_deg
total_area_K

In [ ]:
K_table['hp_idx_o_10', 'ferr_K_min']

fig, ax = plt.subplots()

good = np.log10(np.array(K_table['ferr_K_min'])*1.e-6) > -20
good &= np.log10(np.array(K_table['ferr_K_min'])*1.e-6) <20

cells, fluxes = np.histogram(np.log10(np.array(K_table['ferr_K_min'][good ])*1.e-6), bins=100)
ax.plot(fluxes[1:],
         np.cumsum(cells)*total_area_K/cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('Log$_{10}$ ( Mean K or Ks flux error [Jy] )')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-6.5,-4.5])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.4 * column_width_cm
hieght_cm = width_cm / 1.618
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)
plt.tight_layout()

plt.savefig('./figs/K_cumulative_area_depth.pdf')
plt.savefig('./figs/K_cumulative_area_depth.png')

In [ ]:
K_p25, K_p50, K_p75 = np.nanpercentile(K_table['ferr_K_min'][good ] * 3.,[25, 50, 75])
"""
The K band has coverage over {} square degrees with  3 sigma 
depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.""".format(total_area_K, K_p25, K_p50, K_p75)

And in mags as per reviewer request

In [ ]:
K_table['hp_idx_o_10', 'ferr_K_min']

fig, ax = plt.subplots()

good = np.log10(np.array(K_table['ferr_K_min'])*1.e-6) > -20
good &= np.log10(np.array(K_table['ferr_K_min'])*1.e-6) <20

#cells, fluxes = np.histogram(np.log10(np.array(K_table['ferr_K_min'][good ])*1.e-6), bins=100)

K_pix_mag_depths, _ = flux_to_mag(np.array(K_table['ferr_K_min'][good ])*1.e-6*5)
cells, fluxes = np.histogram(K_pix_mag_depths, bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)



ax.plot(fluxes[1:],
         np.cumsum(cells)*total_area_K/cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('5$\sigma$ $K$ or $Ks$ depth [mag]')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([22,19])
ax.set_ylim([0,1200])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/K_cumulative_area_depth_mag.pdf', bbox_inches='tight')
plt.savefig('./figs/K_cumulative_area_depth_mag.png', bbox_inches='tight')

In [ ]:
K_p25, K_p50, K_p75 = np.nanpercentile(K_pix_mag_depths,[25, 50, 75])
("The K or Ks band has coverage over {} square degrees with  5 sigma"  +
"depths at the 25th, 50th and 75th percentiles of {}, {}, {} respectively.").format(total_area_K, K_p25, K_p50, K_p75)

## Make a table of bands available on each field

In [ ]:
bands = ['u', 'g', 'r', 'i', 'z', 'y',
        'J', 'H', 'K', 'Ks',
        'i1', 'i2', 'i3', 'i4']

In [ ]:
spaced_list = ''
for m, band in enumerate(bands):
    specific_bands = [f for f in filters if f.split('_')[1] == band.lower()]
    for specific_band in specific_bands:
        spaced_list = spaced_list + ', ferr_ap_' + specific_band + '_mean'
        
spaced_list = spaced_list.replace(' ferr_ap_lbc_u_mean,', '')
spaced_list = spaced_list.replace(' ferr_ap_lbc_y_mean,', '')

In [ ]:
spaced_list

In [ ]:

query = """
SELECT DISTINCT hp_idx_O_10{}
FROM depth.main
""".format(spaced_list)
job = service.submit_job(query)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
print(time.time())
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(50) #wait ten seconds and try again
    
print(job.phase)
print('Fetching result', time.time())
table = job_result.fetch_result()

In [ ]:
depth_table = table.table
depth_table 

In [ ]:
cosmos_coverage = Table.read('../../../dmu1/dmu1_ml_COSMOS/data/depths_cosmos_20190402.fits')

In [ ]:
cosmos_coverage[:10].show_in_notebook()

In [ ]:
np.sum(~np.isnan(cosmos_coverage['ferr_irac_i1_mean']))

In [ ]:
plt.hist(cosmos_coverage[~np.isnan(cosmos_coverage['ferr_irac_i1_mean'])]['ferr_irac_i1_mean'], bins=100)

In [ ]:
#We use total here because apertur fluxes are absent
cosmos_coverage['ferr_irac_i1_mean'].name = 'ferr_ap_irac_i1_mean'
cosmos_coverage['ferr_irac_i2_mean'].name = 'ferr_ap_irac_i2_mean'
cosmos_coverage['ferr_irac_i3_mean'].name = 'ferr_ap_irac_i3_mean'
cosmos_coverage['ferr_irac_i4_mean'].name = 'ferr_ap_irac_i4_mean'

In [ ]:
cosmos_coverage['hp_idx_O_10'].name='hp_idx_o_10'

In [ ]:
len(cosmos_coverage)

In [ ]:
len(np.unique(cosmos_coverage['hp_idx_o_10']))

In [ ]:
cosmos_coverage.sort('hp_idx_o_10')
is_duplicate = cosmos_coverage['hp_idx_o_10'][1:] == cosmos_coverage['hp_idx_o_10'][:-1]
#np.append(is_duplicate,False)
cosmos_coverage = cosmos_coverage[~np.append(is_duplicate,False)]

In [ ]:
len(cosmos_coverage)

In [ ]:
shared_cols = set(cosmos_coverage.colnames).intersection(set(depth_table.colnames))

In [ ]:
shared_cols

In [ ]:
cosmos_pixels = np.in1d(depth_table['hp_idx_o_10'], np.unique(cosmos_coverage['hp_idx_o_10']))

In [ ]:
np.sum(cosmos_pixels)

In [ ]:
len(depth_table)

In [ ]:
depth_table= depth_table[~cosmos_pixels]
len(depth_table)

In [ ]:

depth_table = vstack([depth_table, cosmos_coverage[list(shared_cols)]])
len(depth_table)

In [ ]:

for col in depth_table.colnames:
    if depth_table[col].dtype == 'float64':
        depth_table[col].fill_value = np.nan
    if col.startswith('ferr'):
        depth_table[col][depth_table[col] < 1.e-3] *= 1.e6 
    
depth_table = depth_table.filled()
coverage = Table()
coverage.add_column(Column(data=depth_table['hp_idx_o_10'], name='hp_idx_o_10'))
for band in bands:
    specific_bands = [f for f in filters if f.split('_')[1] == band.lower()]
    print(specific_bands)
    for specific_band in specific_bands:
        if 'ferr_ap_{}_mean'.format(specific_band) not in depth_table.colnames:
            specific_bands.remove(specific_band)
    print(specific_bands)
    lowest_depths = np.nanmin([depth_table[column] for column in 
                               ['ferr_ap_{}_mean'.format(s) for s in specific_bands]]
                              , axis=0)
    
    coverage.add_column(Column(data=lowest_depths, name='ferr_ap_{}_mean_min'.format(band)))

In [ ]:
coverage

In [ ]:
#for m, band in enumerate(bands):
#    field_coverages.add_column(Column(data=np.full(len(field_coverages), np.nan), name='ferr_ap_{}_mean_median'.format(band)))

fields_table = Table()
for n, field in enumerate(fields['fields']):
    field_moc = MOC(filename=field['region'].replace('dmu_products/', '../../../'))
    field_cells= Table()
    field_cells.add_column(Column(data=list(field_moc.flattened(order=10)), name='hp_idx_o_10'))
    field_cells.add_column(Column(data=np.full(len(field_cells), field['name'] ), name='field'))
    fields_table = vstack([fields_table, field_cells])
                           


In [ ]:
coverage = join(coverage, fields_table, join_type='left', keys='hp_idx_o_10')

In [ ]:
coverage.write('./data/coverage_band_overview.fits', overwrite=True)

In [ ]:
coverage = Table.read('./data/coverage_band_overview.fits')

In [ ]:
coverage

## Make field summaries

In [ ]:
order10_area = MOC(order=10, cells=[1]).area_sq_deg
order10_area

In [ ]:
#Make a list of all bands
#this was too slow so tried to run whole query at once
field_coverages = Table()
field_coverages.add_column(Column(data=[field['name'] for field in fields['fields']], name='field' ))

for m, band in enumerate(bands):
    field_coverages.add_column(Column(data=np.full(len(field_coverages), np.nan), 
                                      name='{}_band_area'.format(band)))
    field_coverages.add_column(Column(data=np.full(len(field_coverages), np.nan), 
                                      name='ferr_ap_{}_mean_meadian'.format(band)))


for n, field in enumerate(fields['fields']):
    in_field = coverage['field'] == field['name']
    for m, band in enumerate(bands):
        has_band = in_field & ~np.isnan(coverage['ferr_ap_{}_mean_min'.format(band)])
        area = np.sum(has_band) * order10_area
        
        field_coverages['{}_band_area'.format(band)][field_coverages['field'] == field['name']] = area
        
        if np.sum(has_band) != 0:
            depth_median = np.nanmedian(coverage['ferr_ap_{}_mean_min'.format(band)][has_band])
            depth_max = np.nanmax(coverage['ferr_ap_{}_mean_min'.format(band)][has_band])
        else:
            depth_median = np.nan
            depth_max = np.nan
        
        field_coverages['ferr_ap_{}_mean_meadian'.format(band)][field_coverages['field'] == field['name']] = depth_median
        
        if band == 'i1':
            print("Field: {}, {} band coverage: {}, median depth = {}, max depth = {}".format(field['name'],
                                                       band,
                                                       area,
                                                       depth_median,
                                                       depth_max))
        
        
            

In [ ]:
field_coverages

In [ ]:
field_coverages['field', 'ferr_ap_i1_mean_meadian'][field_coverages['ferr_ap_i1_mean_meadian'] > 10]

In [ ]:
field_coverages.write('./data/field_coverages_band_overview.fits', overwrite=True)

In [ ]:
field_coverages = Table.read('./data/field_coverages_band_overview.fits')

In [ ]:
table = 'field '
for band in bands:
    table += '& \\multicolumn{2}{c}{' + band + '}'
    
table += ' \\\\ \n'

table += '\\cline{1-29} \n'

for band in bands:
    table += '& a & \\sigma '

table += '\\\\ \n'

In [ ]:
len('AKARI-NEP&&&9.6&1.5&9.6&1.6&9.6&1.5&9.6&3.2&9.6&7.2&&&&&&&&&7.2&1.7&7.2&1.4&&&&\\'.split('&'))

In [ ]:
print(table)

In [ ]:


for n, field in enumerate(field_coverages):
    latex = ''
    for element in field:


        if isinstance(element , float):
            if np.isnan(element):
                element = ''
            elif np.isclose(element, 0.0):
                element = ''
            else:
                element = round(element,1)


        
        latex +=  str(element) + '&' 
    print(latex[:-1] + '\\\\')
    table += latex[:-1] + '\\\\ \n'

table += '\\hline'

In [ ]:
print(table)

In [ ]:
round(np.nan,2)

## Plot general overview of depths

In [ ]:
def depths_sample(band):
    mask = ~np.isnan(coverage['ferr_ap_{}_mean_min'.format(band)])
    mask &= coverage['ferr_ap_{}_mean_min'.format(band)] > 0.
    mask &= coverage['ferr_ap_{}_mean_min'.format(band)] <1.e3
    area = (np.sum(mask)/len(coverage)) * 1270.
    return np.log10(np.array(coverage['ferr_ap_{}_mean_min'.format(band)][mask]) *5.e-6 ), area
    

data = [ depths_sample(band)[0] for band in bands ]
areas = [ depths_sample(band)[1] for band in bands ]

In [ ]:
bands

In [ ]:
bands_fir = ['u', 'g', 'r', 'i', 'z', 'y', 'J', 'H', 'K', 'Ks', 'i1', 'i2', 'i3', 'i4', 
             '24', '100', '160', '250', '350', '500']

In [ ]:
en1_depths = Table.read('../../../dmu32/dmu32_ELAIS-N1/data/depths_elais-n1_20171016.fits'
                       )['hp_idx_O_10', 
           'ferr_mips_24_mean',
           'ferr_pacs_green_mean',
           'ferr_pacs_red_mean',
           'ferr_spire_250_mean',
           'ferr_spire_350_mean',
           'ferr_spire_500_mean']
en1_depths['hp_idx_O_10'].name = 'hp_idx_o_10'

In [ ]:
len(en1_depths)

In [ ]:
en1_depths.sort('hp_idx_o_10')
is_duplicate = en1_depths['hp_idx_o_10'][1:] == en1_depths['hp_idx_o_10'][:-1]
#np.append(is_duplicate,False)
en1_coverage = en1_depths[~np.append(is_duplicate,False)]

In [ ]:
len(en1_coverage)

In [ ]:
full_coverage = join(coverage, en1_coverage, join_type='left', keys='hp_idx_o_10')

In [ ]:
full_coverage

In [ ]:
full_names = {
    '24':'mips_24', 
    '100':'pacs_green', 
    '160':'pacs_red', 
    '250':'spire_250', 
    '350':'spire_350', 
    '500':'spire_500'
}
def depths_sample(band):
    if band in ['24', '100', '160', '250', '350', '500']:
        band = full_names[band] 
        mask = ~np.isnan(full_coverage['ferr_{}_mean'.format(band)])
        mask &= full_coverage['ferr_{}_mean'.format(band)] > 0.
        #mask &= full_coverage['ferr_{}_mean'.format(band)] <1.e3
        area = (np.sum(mask)/len(full_coverage)) * 1270. * (1270./14.)
        return np.log10(np.array(full_coverage['ferr_{}_mean'.format(band)][mask]) *5.e-6 ), area
    else:
        mask = ~np.isnan(coverage['ferr_ap_{}_mean_min'.format(band)])
        mask &= full_coverage['ferr_ap_{}_mean_min'.format(band)] > 0.
        mask &= full_coverage['ferr_ap_{}_mean_min'.format(band)] <1.e3
        area = (np.sum(mask)/len(full_coverage)) * 1270.
        return np.log10(np.array(full_coverage['ferr_ap_{}_mean_min'.format(band)][mask]) *5.e-6 ), area
    

data = [ depths_sample(band)[0] for band in bands_fir ]
areas = [ depths_sample(band)[1] for band in bands_fir ]

In [ ]:
len(data)

In [ ]:


# fake data
fs = 10  # fontsize
#pos = [1, 2, 4, 5, 7, 8]
#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('band')

#ax.violinplot(np.array(coverage['ferr_ap_{}_mean_min'.format('g') ] ) )
#ax.set_title('Custom violinplot 1', fontsize=fs)

ax.set_ylabel('log10( 5$\sigma$ Depths  [Jy] )')
set_axis_style(ax, ['$' + band + '$' for band in bands_fir])
ax.set_ylim(-7, 0)



parts = ax.violinplot(data, showmeans=False, showmedians=False,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1)

cax, _ = mpl.colorbar.make_axes(ax)
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('area [square degrees]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/band_depths_overviews.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews.png', bbox_inches='tight')

In [ ]:
widths = [np.max(np.histogram(data[n], bins = 100, density = True)[0])*areas[n] for n in np.arange(len(data)) ]
widths /= np.max(widths)

print(widths)

In [ ]:
# fake data
fs = 10  # fontsize
#pos = [1, 2, 4, 5, 7, 8]
#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('band')

#ax.violinplot(np.array(coverage['ferr_ap_{}_mean_min'.format('g') ] ) )
#ax.set_title('Custom violinplot 1', fontsize=fs)

ax.set_ylabel('log10( 5$\sigma$ Depths  [Jy] )')
set_axis_style(ax, ['$' + band + '$' for band in bands_fir])
ax.set_ylim(-7, 0)


#areas/np.max(areas)
parts = ax.violinplot(data, widths=widths, showmeans=False, showmedians=False,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1)

cax, _ = mpl.colorbar.make_axes(ax)
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('Area [square degrees]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/band_depths_overviews_areaweighted.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews_areaweighted.png', bbox_inches='tight')

In [ ]:
# fake data
fs = 10  # fontsize
#pos = [1, 2, 4, 5, 7, 8]
#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('band')

#ax.violinplot(np.array(coverage['ferr_ap_{}_mean_min'.format('g') ] ) )
#ax.set_title('Custom violinplot 1', fontsize=fs)

ax.set_ylabel('log10( 5$\sigma$ Depths  [Jy] )')
set_axis_style(ax, ['$' + band + '$' for band in bands_fir])
ax.set_ylim(-7, 0)



parts = ax.violinplot(data, widths=np.log10(areas)/np.log10(np.max(areas)), showmeans=False, showmedians=False,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1)

cax, _ = mpl.colorbar.make_axes(ax)
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('area [square degrees]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/band_depths_overviews_logareaweighted.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews_logareaweighted.png', bbox_inches='tight')

## Add SED to plot

In [ ]:
#s.add_contribution("HELP_SED", orig_spec['WAVE'], orig_spec['LUMIN'])

In [ ]:
cigale_filternames = {
    'u':  'u_prime',
    'g':  'MCam_g',
    'r':  'MCam_r',
    'i':  'MCam_i',
    'z':  'MCam_z',
    'y':  'WFCAM_Y',
    'J':  'WFCAM_J',
    'H':  'WFI_H',
    'K':  'WFI_K',
    'Ks': 'Ks_2mass',
    'i1': 'IRAC1',
    'i2': 'IRAC2',
    'i3': 'IRAC3',
    'i4': 'IRAC4'
}

In [ ]:



gal1 = './data/HELP_J095946.083p021914.438_best_model.fits'
gal2 = './data/HELP_J100130.443p020929.494_best_model.fits'
gal3 = './data/HELP_J095809.302p013203.775_best_model.fits'
gal4 = './data/HELP_J095822.986p013145.336_best_model.figs'
gal5 = './data/HELP_J003412.527-441056.846_best_model.fits'  #z=4
gal6 = './data/HELP_J095738.934+021508.530_best_model.fits' #From hedam - high fluxes
gal7 = './data/HELP_J095818.598+013057.910_best_model.fits'
gal8 = './data/HELP_J095809.488+013225.513_best_model.fits'
gal9 = './data/HELP_J100108.952+022730.528_best_model.fits'
orig_spec = Table.read(gal9)
        
s = SED()
    # This is wrong because the best SED we get from CIGALE is redshifted (written by Yannick)
s.add_contribution("HELP_SED", orig_spec['wavelength'], orig_spec['L_lambda_total'])
    
#z=1
zs = np.arange(1, 5, 1)




In [ ]:
gal_fluxes = np.full([len(zs), len(bands)], np.nan)

In [ ]:
fig, ax = plt.subplots()
for n, z in enumerate(zs):
    print('z = {}:'.format(z))
    sed = s.copy()
    mod = get_module("redshifting", redshift=z)
    mod.process(sed)
    ax.plot(np.log10(sed.wavelength_grid), np.log10(sed.fnu * 1.e-3), label=str(z))
#ax.plot(np.log10(orig_spec['wavelength']), np.log10(orig_spec['Fnu']))

plt.legend()
ax.set_ylim(-7, 0)
#ax.set_xlim(0, 14.75)

In [ ]:
pos = [3561.,     #u  (SDSS)
       4866.,     #g  (GPC1)
       6215.,     #r  (GPC1)
       7545.,     #i  (GPC1)
       8680.,     #z  (GPC1)
       9633.,     #y  (GPC1)
       12510.,    #J  (UKIRT)
       16377.,    #H  (UKIRT)
       22081.,    #K   (UKIRT)
       21496.,    #Ks (WIRCam)
       36000.,    #i1
       45000.,    #i2 
       56000.,    #i3
       80000.,    #i4
       240000.,    #mips_24
       1000000.,    #pacs_100
       1600000.,    #pacs_160
       2500000.,    #spire_250
       3500000.,    #spire_350
       5000000.,    #spire_500
      ]   
fwhms = [
       [3048., 4028.],     #u  (SDSS)
       [3943., 5593.],     #g  (GPC1)
       [5386., 7036.],     #r  (GPC1)
       [6778., 8304.],     #i  (GPC1)
       [8028., 9346.],     #z  (GPC1)
       [9100., 10838.],     #y  (GPC1)
       [11690., 13280],     #J  (UKIRT)
       [14920., 17840.],     #H  (UKIRT)
       [20290., 23800.],     #K   (UKIRT)
       [19578., 23431.],     #Ks (WIRCam)
       [31296, 39614 ],     #i1
       [39173, 50561],      #i2 
       [48983, 65089],     #i3
       [62994, 95876],      #i4
       [240000.,340000.],    #mips_24 GUESSES!!!!
       [1000000.,1500000.],    #pacs_100
       [1600000.,1900000.],    #pacs_160
       [2500000.,3000000.],    #spire_250
       [3500000.,4000000.],    #spire_350
       [5000000.,5500000.]    #spire_500
        ]

In [ ]:
['${}$'.format(band).replace(
    '$Ks$', '').replace(
    'K', 'K/Ks').replace(
     '100', 'PACS 100/160').replace(
     '160', ' ').replace(
     '250', 'SPIRE 250/350/500').replace(
     '350', ' ').replace(
     '500', ' ') for band in bands_fir]

In [ ]:
# fake data
fs = 10  # fontsize
# Angstroms     


#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

ax2 = ax.twiny()

#ax2.set_xlim(3000., 100000)
#ax2.set_xticks(pos)
#ax2.set_xticklabels(['$' + band + '$' for band in bands])
#ax2.set_xlabel('band')
#ax2.set_xscale('log')
ax2.set_xlim(np.log10(3000.), np.log10(10000000))
ax2.set_xticks(np.log10(pos))
ax2.set_xticklabels(['$u$',
 '$g$',
 '$r$',
 '$i$',
 '$z$',
 '$y$',
 '$J$',
 '$H$',
 '  $K/Ks$',
 '',
 '$i1$',
 '$i2$',
 '$i3$',
 '$i4$',
 'MIPS 24',
 '       PACS',
 '',
 '           SPIRE',
 '',
 ''])
ax2.set_xlabel('band')
#ax2.set_xscale('log')


line_styles = [':', '-.', '--', '-']
colours = ['y', 'b', 'g', 'r', 'k']
for n, z in enumerate([0.25, 1, 2, 3, 4]):
    sed = s.copy()
    mod = get_module("redshifting", redshift=z)
    mod.process(sed)
    ax.plot(sed.wavelength_grid*10, 
            np.log10(sed.fnu * 1.e-3),
            c='k',
            #c= colours[n],
            #linestyle = line_styles[n],
            label= 'z = {}'.format(z),
            linewidth=1.0,
            alpha=1.
           )
    for m, band in enumerate(cigale_filternames):
        continue
        if m == 0:
            lab = 'z = {}'.format(z)
        else:
            lab=None
        ax.plot([fwhms[m][0], fwhms[m][1]], [np.log10(gal_fluxes[n, m]   )-3, 
                                     np.log10(gal_fluxes[n, m]   )-3], 
               # c='k',
                c= colours[n],
               #linestyle = line_styles[n],
               label=lab,
                alpha=1.0
               )

        




ax.set_ylabel('log10(Depths [Jy] )')

#ax.set_xticks(pos)
#ax.set_xticklabels(['${}$'.format(band.replace('Ks', '').replace('K', 'K/Ks')) for band in bands])
ax.set_xlim(3000., 10000000)
ax.set_xscale('log')    
ax.set_xlabel('Wavelength [nm]')
ax.set_ylim(-7, 0)
#ax.legend(loc=2)


#widths
log_widths = np.ones(len(pos)) * (pos) * .2# * widths
#log_widths = np.ones(len(pos)) * 1000.
#areas/np.max(areas)
parts = ax.violinplot(data, 
                      positions=pos, 
                      widths=log_widths, 
                      showmeans=False, 
                      showmedians=False, #widths=widths,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1.0)

cax, _ = mpl.colorbar.make_axes([ax, ax2])
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('Area [square degrees]')



plt.figtext(0.335, 0.6, 'z=0.25')
plt.figtext(0.38, 0.45, 'z=1')
plt.figtext(0.4, 0.38, 'z=2')
plt.figtext(0.44, 0.35, 'z=3')
plt.figtext(0.45, 0.27, 'z=4')





plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave.png', bbox_inches='tight')

In [ ]:
#take the median luminosity in a redshift bin?
#nu Lnu??

In [ ]:
log_widths

In [ ]:
# fake data
fs = 10  # fontsize
# Angstroms     


#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

ax2 = ax.twiny()

#ax2.set_xlim(3000., 100000)
#ax2.set_xticks(pos)
#ax2.set_xticklabels(['$' + band + '$' for band in bands])
#ax2.set_xlabel('band')
#ax2.set_xscale('log')
ax2.set_xlim(np.log10(300.), np.log10(1000000))
ax2.set_xticks(np.log10(np.array(pos)/10))
ax2.set_xticklabels(['${}$'.format(band).replace('$Ks$', '').replace('K', 'K/Ks') for band in bands])
ax2.set_xlabel('band')
#ax2.set_xscale('log')


line_styles = [':', '-.', '--', '-']
colours = ['y', 'b', 'g', 'r', 'k']
for n, z in enumerate([0.25, 1, 2, 3, 4]):
    sed = s.copy()
    mod = get_module("redshifting", redshift=z)
    mod.process(sed)
    ax.plot(sed.wavelength_grid, #*10 factor ten turns angstrom to nm fix unit error
            #np.log10(sed.fnu * 1.e-3), #This is for flux in Jy
            flux_to_mag(sed.fnu * 1.e-3)[0], #This is for flux in mag
            c='k',
            #c= colours[n],
            #linestyle = line_styles[n],
            label= 'z = {}'.format(z),
            linewidth=1.0,
            alpha=1.
           )
    for m, band in enumerate(cigale_filternames):
        continue
        if m == 0:
            lab = 'z = {}'.format(z)
        else:
            lab=None
        ax.plot([fwhms[m][0], fwhms[m][1]], [np.log10(gal_fluxes[n, m]   )-3, 
                                     np.log10(gal_fluxes[n, m]   )-3], 
               # c='k',
                c= colours[n],
               #linestyle = line_styles[n],
               label=lab,
                alpha=1.0
               )

        




ax.set_ylabel('5$\sigma$ depth [mag]')

#ax.set_xticks(pos)
#ax.set_xticklabels(['${}$'.format(band.replace('Ks', '').replace('K', 'K/Ks')) for band in bands])
ax.set_xlim(300., 1000000)
ax.set_xscale('log')    
ax.set_xlabel('Wavelength [nm]')
ax.set_ylim(27, 16)
#ax.legend(loc=2)


#widths
log_widths = np.ones(len(pos)) * (pos) * .2 * widths
#areas/np.max(areas)
parts = ax.violinplot([flux_to_mag(10**d)[0] for d in data], #flux_to_mag assumes Jy
                      positions=np.array(pos)/10, #fix unit error 
                      widths=log_widths/10, 
                      showmeans=False, 
                      showmedians=False, #widths=widths,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1.0)

cax, _ = mpl.colorbar.make_axes([ax, ax2])
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('Area [deg.$^2$]')



plt.figtext(0.49, 0.81, 'z = 0.25')
plt.figtext(0.70, 0.645, 'z = 1')
plt.figtext(0.70, 0.52, 'z = 2')
plt.figtext(0.70, 0.435, 'z = 3')
plt.figtext(0.70, 0.385, 'z = 4')





plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave_mag.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave_mag.png', bbox_inches='tight')

In [ ]:
['${}$'.format(band).replace('$Ks$', '').replace('K', 'K/Ks') for band in bands]

In [ ]:
data[0].shape

In [ ]:


fig, ax = plt.subplots()
#cmap = mpl.cm.get_cmap('viridis', 256)
#norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.min(areas))

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

#cb = mpl.colorbar.ColorbarBase(ax, cmap=cmap,  norm=norm)

#cmap = mpl.colors.ListedColormap('viridis')
#colors = cmap(np.array(areas))


im = plt.scatter(np.arange(len(areas)), areas, c=colors)


cax, _ = mpl.colorbar.make_axes(ax)
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)
cbar.set_label('area [square degrees]')
#cax.set_yticklabels(['$-\pi$', '0', '$\pi$'])
#cb = mpl.colorbar.ColorbarBase(ax, cmap=cmap,  norm=norm)
# Optionally add a colorbar
#cax, _ = mpl.colorbar.make_axes(ax)
#cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)



In [ ]:
colors

In [ ]:
cmap = mpl.cm.hot
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)  # The color is the angle
#colors[:, 3] = 0.5
print(scalmap.to_rgba(500),
     scalmap.to_rgba(1000),
     scalmap.to_rgba(100))
colors

In [ ]:
mask = ~np.isnan(coverage['ferr_ap_{}_mean_min'.format('i1')])
mask |= coverage['ferr_ap_{}_mean_min'.format('i1')] > 0.
mask |= coverage['ferr_ap_{}_mean_min'.format('i1')] <1.e3
plt.hist(np.log10(np.array(coverage['ferr_ap_{}_mean_min'.format('i1')][mask ]) *1.e-6), bins=50)

In [ ]:
areas

In [ ]:
np.sum(np.log10(coverage['ferr_ap_{}_mean_min'.format('i1')] )  >3)

In [ ]:
np.sum(coverage['ferr_ap_{}_mean_min'.format('i1')]  >10.)

## All field comparison

In [ ]:
def depths_sample_field(band, field):
    
    mask = ~np.isnan(coverage['ferr_ap_{}_mean_min'.format(band)])
    mask &= coverage['ferr_ap_{}_mean_min'.format(band)] > 0.
    mask &= coverage['ferr_ap_{}_mean_min'.format(band)] <1.e3
    mask &= coverage['field'] == field
    area = (np.sum(mask)/len(coverage[coverage['field'] == field])) #* 1270. #Return fractional areas
    
    pixel_depths = np.log10(np.array(coverage['ferr_ap_{}_mean_min'.format(band)][mask]) *5.e-6 )
    if np.sum(mask)== 0:
        pixel_depths = np.full(100, -9.)
    return pixel_depths, area
    
f = 'ELAIS-N1'
data_field = [ depths_sample_field(band, f)[0] for band in bands ]
areas_field = [ depths_sample_field(band, f)[1] for band in bands ]

In [ ]:
areas_field

In [ ]:
dim = [4,6]
fig, axes = plt.subplots(dim[1], dim[0], sharex=True, sharey=True)
plt.rcParams.update({'font.size': 12})
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=0, vmax=1)
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)

for n, f in enumerate(np.unique(coverage['field'])):
        
    x, y = np.floor_divide(n, dim[0]), np.remainder(n, dim[0])
    
    
    #axes[x,y].scatter( np.arange(20), np.arange(20))
    #axes[x,y].legend()
    
    
    #####MAKE FIELD DATA#####
    data_field = [ depths_sample_field(band, f)[0] for band in bands ]
    areas_field = [ depths_sample_field(band, f)[1] for band in bands ]
    data_field_mag = [flux_to_mag(10**d)[0] for d in data_field]
    
    
    colors = scalmap.to_rgba(areas_field)   # The color is the angle
    colors[:, 3] = 1

   

    def set_axis_style(ax, labels):
        axes[x,y].get_xaxis().set_tick_params(direction='out')
        axes[x,y].xaxis.set_ticks_position('bottom')
        axes[x,y].tick_params(axis='x', labelsize=8)
        axes[x,y].set_xticks(np.arange(1, len(labels) + 1))
        axes[x,y].set_xticklabels(labels)
        axes[x,y].set_xlim(0.25, len(labels) + 0.75)
        #axes[x,y].set_xlabel('band')
        #axes[x,y].set_ylabel('5$\sigma$ depth [mag]')
    #ax.violinplot(np.array(coverage['ferr_ap_{}_mean_min'.format('g') ] ) )
    #ax.set_title('Custom violinplot 1', fontsize=fs)

    #axes[x,y].set_ylabel('log10( 5$\sigma$ Depths  [Jy] )')
    set_axis_style(axes[x,y], ['$' + band + '$' for band in bands])
    axes[x,y].set_ylim(28, 17)



    parts = axes[x,y].violinplot(data_field_mag, widths=0.75, showmeans=False, showmedians=False,
            showextrema=False)
    
    
    #This is just to add the field name
    axes[x,y].scatter([-99],[-99], 
                      label=f.replace('SGP', 'HATLAS-SGP').replace('NGP', 'HATLAS-NGP'), 
                      c='w', s=0.0001)
    axes[x,y].legend(frameon=False, loc=4)

    for n, part in enumerate(parts['bodies']):
        part.set_facecolor(colors[n])
        part.set_alpha(1)

#cax, _ = mpl.colorbar.make_axes(axes[dim[1]-1,dim[0]-1], panchor=(-8.8, 10.6)) #5 3 depend on 4 x 6
#This bit moves the color bar
cax = inset_axes(axes[dim[1]-1,dim[0]-1],
                   width="5%",  # width = 5% of parent_bbox width
                   height="85%",  # height : 50%
                   loc=3,
                   bbox_to_anchor=(0.05, 0.08, 1, 1),
                   bbox_transform=axes[dim[1]-1,dim[0]-1].transAxes,
                   borderpad=0,
                   )

ticks = [0, 0.5, 1]
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels(ticks)
cbar.set_label('coverage')
    
axes[dim[1]-1,dim[0]-1].tick_params(axis='x', labelsize=8)
#axes[dim[1]-1,dim[0]-1].set_xlabel('band')
    
fig.text(0.5, 0.07, 'Bands', ha='center')
fig.text(0.04, 0.5, '5$\sigma$ depth [mag]', va='center', rotation='vertical')

fig.set_size_inches(10, 12)
fig.subplots_adjust(hspace=0, wspace=0)

plt.rc('axes', labelsize=12)
plt.savefig('./figs/fields_depths_comparison_grid.pdf', bbox_inches='tight')
plt.savefig('./figs/fields_depths_comparison_grid.png', bbox_inches='tight')

## Summarise the filters

In [ ]:
for band in bands:
    specific_bands = [f for f in filters if f.split('_')[1] == band.lower()]
    print(specific_bands)
    for specific_band in specific_bands:
        if 'ferr_ap_{}_mean'.format(specific_band) not in coverage.colnames:
            specific_bands.remove(specific_band)
    #print(specific_bands)

In [ ]:
filter_info = Table.read(herschelhelp_python_loc + 'documentation/filters.csv')

In [ ]:
has_broad_typical = np.full(len(filter_info), False)
for band in bands:
    has_broad_typical |= filter_info['filter'] == band
    
filter_info[has_broad_typical]

In [ ]:
has_optical = np.full(len(filter_info), False)
optical_bands = ['u', 'g', 'r', 'i', 'z', 'y']
has_near_infrared = np.full(len(filter_info), False)
near_infrared_bands = ['J', 'H', 'K', 'Ks']

for band in bands:
    if band in optical_bands:
        has_optical |= filter_info['filter'] == band
    if band in near_infrared_bands:
        has_near_infrared |= filter_info['filter'] == band

In [ ]:
def get_bands(camera):
    band_list = []
    for b in filter_info['filter'][filter_info['camera'] == camera]:
        #print(b)
        band_list.append( b)
    band_list_ordered_string = ''
    for band in bands:
        if band in band_list:
            band_list_ordered_string += band
    
    return '$' + band_list_ordered_string + '$'



print(get_bands('HyperSuprimeCam'))

In [ ]:


optical_sentence = ''
for camera in np.unique(filter_info['camera'][has_optical]):
    if camera == 'Standard camera response (camera unknown)':
        continue
    row = filter_info[filter_info['camera'] == camera][0] 
    optical_sentence += 'From {} on {} we have the optical bands {}. '.format(row['camera'].replace('The', 'the'), 
                                                             row['telescope'].replace('The', 'the'),
                                                             get_bands(camera))
    
print(optical_sentence)

In [ ]:
near_infrared_sentence = ''
for camera in np.unique(filter_info['camera'][has_near_infrared]):
    if camera == 'Standard camera response (camera unknown)':
        continue
    row = filter_info[filter_info['camera'] == camera][0] 
    near_infrared_sentence += 'From {} on {} we have the optical bands {}. '.format(row['camera'].replace('The', 'the'), 
                                                             row['telescope'].replace('The', 'the'),
                                                             get_bands(camera))
    
print(near_infrared_sentence)

## Count the number of flagged objects

For each band lets count the number of objects that have a flag

# Make cumulative 1 d depths for MIPS, PACS, SPIRE

In [ ]:
np.array(en1_depths['ferr_mips_24_mean'])[~np.isnan(en1_depths['ferr_mips_24_mean'])]

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

mips_pix_mag_depths = np.array(np.log10(en1_depths['ferr_mips_24_mean']*1.e-6 ))
cells, fluxes = np.histogram(mips_pix_mag_depths[~np.isnan(mips_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*200./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ MIPS 24 depth [Jy]) ')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-5,-4.5])
ax.set_ylim([0,210])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/mips_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/mips_cumulative_area_depth.png', bbox_inches='tight')

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

pacs_pix_mag_depths = np.array(np.log10(en1_depths['ferr_pacs_green_mean']*1.e-6 ))
cells, fluxes = np.histogram(pacs_pix_mag_depths[~np.isnan(pacs_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*200./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ PACS 100 depth [Jy]) ')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-2.2,-1.5])
ax.set_ylim([0,210])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/pacs_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/pacs_cumulative_area_depth.png', bbox_inches='tight')

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

spire_pix_mag_depths = np.array(np.log10(en1_depths['ferr_spire_250_mean']*1.e-6 ))
cells, fluxes = np.histogram(spire_pix_mag_depths[~np.isnan(spire_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*1100./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ SPIRE 250 depth [Jy])')
ax.set_ylabel('Cumulative area [deg.$^2$]')
#ax.set_xlim([-2.2,-1.5])
ax.set_ylim([0,1200])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/spire_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/spire_cumulative_area_depth.png', bbox_inches='tight')

# 2d depth plots

Make a cumulative 2d area that is at least as deep.

In [ ]:
en1_d = Table.read('../../../dmu32/dmu32_ELAIS-N1/data/depths_elais-n1_20171016.fits')

In [ ]:
fig, ax = plt.subplots()
good = ~np.isnan(en1_d['ferr_ap_irac_i1_mean']) & ~np.isnan(en1_d['ferr_mips_24_mean'])
good &= en1_d['ferr_ap_irac_i1_mean'] < 1.
good &= en1_d['ferr_mips_24_mean'] > 12.
good &= en1_d['ferr_mips_24_mean'] < 18.
ax.hist2d(en1_d['ferr_ap_irac_i1_mean'][good],en1_d['ferr_mips_24_mean'][good],  bins=25)


In [ ]:
good &= en1_d['ferr_ap_irac_i1_mean'] < 1.
good &= en1_d['ferr_mips_24_mean'] > 12.
good &= en1_d['ferr_mips_24_mean'] < 18.

hist = np.histogram2d(en1_d['ferr_ap_irac_i1_mean'][good],en1_d['ferr_mips_24_mean'][good],  bins=25)
hist

In [ ]:
vals

In [ ]:
from pymoc import MOC
t_moc = MOC(13, [en1_d['hp_idx_O_13'][0] ])
t_moc.area_sq_deg

In [ ]:
band1 = ['ferr_ap_irac_i1_mean',0.,1.]
band2 = ['ferr_mips_24_mean',0.,20. ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=1000)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([0.5, 1.])
ax.set_ylim([12., 19.])

ax.set_xlabel('IRAC $i1$ Depth [$\mu$Jy]')
ax.set_ylabel('MIPS 24$\mu$m Depth [$\mu$Jy]')


plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_i1_mips_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_i1_mips_depth.png', bbox_inches='tight')

In [ ]:
band1 = ['ferr_ap_r_mean',0.,1.2]
band2 = ['ferr_ap_k_mean',0.,1.1 ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=100)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([0.05, .07075])
ax.set_ylim([0.3, 1.1])

ax.set_xlabel('Deepest $r$ Depth [$\mu$Jy]')
ax.set_ylabel('Deepest $K$ or $K_s$ Depth [$\mu$Jy]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_r_k_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_r_k_depth.png', bbox_inches='tight')

In [ ]:
np.sum(vals)

In [ ]:
band1 = ['ferr_pacs_green_mean',0.,40000.2]
band2 = ['ferr_spire_350_mean',0.,4500.1 ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=100)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([9000., 40000.])
ax.set_ylim([2000., 4500.])

ax.set_xlabel('PACS 100$\mu$m Depth [$\mu$Jy]')
ax.set_ylabel('SPIRE 250$\mu$m Depth [$\mu$Jy]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_pacs_spire_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_pacs_spire_depth.png', bbox_inches='tight')